# Python API Challenge: Vacation destinations map

In [47]:
import matplotlib.pyplot as plt
import json
import pandas as pd
import numpy as np
import requests
import gmaps
import os

In [3]:
# Import weather data csv
weather_df = pd.read_csv("../WeatherPy/WeatherData.csv", index_col = False)
weather_df = weather_df.drop(columns = "Unnamed: 0")
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,tilichiki,60.4667,166.1000,56.48,83,100,1.32,RU,1626294034
1,conceicao do araguaia,-8.2578,-49.2647,94.19,22,1,2.98,BR,1626294034
2,sorong,-0.8833,131.2500,78.64,85,100,6.17,ID,1626293747
3,macau,22.2006,113.5461,87.08,79,20,8.05,MO,1626294035
4,portales,34.1862,-103.3344,88.11,28,8,16.33,US,1626294035


In [53]:
# Search for places that had temperatures between 70 and 80 F, winds below 10mph, and few clouds
vacation_df = weather_df.loc[weather_df["Max Temp"] > 70]
vacation_df = vacation_df.loc[vacation_df["Max Temp"] < 80]
vacation_df = vacation_df.loc[vacation_df["Wind Speed"] < 10]
vacation_df = vacation_df.loc[vacation_df["Cloudiness"] < 10]

vacation_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,letnyaya stavka,45.4306,43.4472,76.21,30,0,3.83,RU,1626294037
17,novopokrovka,49.8425,36.5477,75.58,52,0,0.94,UA,1626294202
122,petrovskaya,45.4314,37.9550,72.09,43,0,9.17,RU,1626302836
142,dawlatabad,36.4141,64.9053,78.80,21,1,2.44,AF,1626302937
143,ponta do sol,32.6667,-17.1000,72.72,84,1,4.36,PT,1626302938
159,fez,34.0372,-4.9998,79.05,44,0,3.44,MA,1626302942
173,mandaguacu,-23.3472,-52.0953,71.20,46,0,5.75,BR,1626303026
178,roma,43.2128,-75.4557,79.86,78,1,5.75,US,1626303027
180,zhob,31.3411,69.4481,74.70,72,2,2.13,PK,1626303028
186,khomeyn,33.6423,50.0789,74.12,18,0,3.94,IR,1626303029


In [5]:
# Heat map of locations in WeatherData.csv with conditions matching the criteria above, weighted by humidity
locations = vacation_df[["Lat", "Lng"]]
humidity = vacation_df["Humidity"]

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [45]:
lat = vacation_df.loc[vacation_df["City"] == "roma"]["Lat"].astype(float)
lng = vacation_df.loc[vacation_df["City"] == "roma"]["Lng"].astype(float)
print(f"lat= {lat}, lng= {lng}")

lat= 178    43.2128
Name: Lat, dtype: float64, lng= 178   -75.4557
Name: Lng, dtype: float64


In [38]:
vacation_df.loc[vacation_df["City"] == "roma"]

178    43.2128
Name: Lat, dtype: float64

In [24]:
vacation_df.dtypes

City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

In [57]:
# Search for hotels within 5000m of each city using Google Places API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Add a column to store the name of the nearest hotel
hotel_df = vacation_df.copy()
hotel_df["Hotel Name"] = ""

params = {
    "rankby": "distance",
    "keyword": "hotel",
    "key": gkey
}

for index, row in vacation_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    try:
        response = requests.get(base_url, params = params).json()
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except IndexError:
        hotel_df.loc[index, "Hotel Name"] = 0

In [61]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
13,letnyaya stavka,45.4306,43.4472,76.21,30,0,3.83,RU,1626294037,"Gostinitsa ""Grand Otel'"""
17,novopokrovka,49.8425,36.5477,75.58,52,0,0.94,UA,1626294202,Hotel Meduza
122,petrovskaya,45.4314,37.9550,72.09,43,0,9.17,RU,1626302836,Хостел АПЕЛЬСИН
142,dawlatabad,36.4141,64.9053,78.80,21,1,2.44,AF,1626302937,0
143,ponta do sol,32.6667,-17.1000,72.72,84,1,4.36,PT,1626302938,Hotel Enotel Baía
159,fez,34.0372,-4.9998,79.05,44,0,3.44,MA,1626302942,Splendid Hôtel
173,mandaguacu,-23.3472,-52.0953,71.20,46,0,5.75,BR,1626303026,Hotel Cadori
178,roma,43.2128,-75.4557,79.86,78,1,5.75,US,1626303027,Quality Inn Rome- Verona
180,zhob,31.3411,69.4481,74.70,72,2,2.13,PK,1626303028,Mehfil hotel
186,khomeyn,33.6423,50.0789,74.12,18,0,3.94,IR,1626303029,Melli Bank


In [62]:
# Drop the city that had no nearby hotel

hotel_df = hotel_df.drop(hotel_df[hotel_df["Hotel Name"] == 0].index)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
13,letnyaya stavka,45.4306,43.4472,76.21,30,0,3.83,RU,1626294037,"Gostinitsa ""Grand Otel'"""
17,novopokrovka,49.8425,36.5477,75.58,52,0,0.94,UA,1626294202,Hotel Meduza
122,petrovskaya,45.4314,37.9550,72.09,43,0,9.17,RU,1626302836,Хостел АПЕЛЬСИН
143,ponta do sol,32.6667,-17.1000,72.72,84,1,4.36,PT,1626302938,Hotel Enotel Baía
159,fez,34.0372,-4.9998,79.05,44,0,3.44,MA,1626302942,Splendid Hôtel
173,mandaguacu,-23.3472,-52.0953,71.20,46,0,5.75,BR,1626303026,Hotel Cadori
178,roma,43.2128,-75.4557,79.86,78,1,5.75,US,1626303027,Quality Inn Rome- Verona
180,zhob,31.3411,69.4481,74.70,72,2,2.13,PK,1626303028,Mehfil hotel
186,khomeyn,33.6423,50.0789,74.12,18,0,3.94,IR,1626303029,Melli Bank
238,agropoli,40.3469,14.9966,73.29,75,0,6.26,IT,1626303160,Villa Ada B&B


In [64]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

fig2 = gmaps.figure()
markers = gmaps.marker_layer(locations = locations,
                  info_box_content = hotel_info)
fig2.add_layer(markers)

fig2

Figure(layout=FigureLayout(height='420px'))